# Best Strategy in Finding a Business Location

To find the best business location, start by analyzing your target market, researching potential areas, and considering factors like accessibility, infrastructure, and zoning regulations.
Here's a more detailed strategy:  

##  1. Define Your Needs and Goals: 
- **Target Market:** Identify your ideal customer base and where they are located
    - For definition see [this](https://www.investopedia.com/terms/t/target-market.asp#:~:text=Demographic%3A%20These%20are%20the%20main,in%20the%20era%20of%20globalization.)
- **Business Type:** Determine if your business requires 
    - high foot traffic
    - proximity to suppliers
    - access to specific infrastructure (e.g., highways, rail yards)
- **Budget:** Establish a realistic budget for 
    - rent
    - utilities
    - other location-related costs
- **Brand Image:** Consider how the location will impact your brand's image and customer perception

### Market Analysis
Example: Laundromat in Vandergrift, PA

In [1]:
business = 'laudromat'
location = 'Vandergrift, PA'

- **Demographic Analysis:** Use local census data to understand the demographics of your area. 
    - Look for age distributions, income levels, and household sizes that match the typical profile of a user (for our example laundromat user).
- **Psychographic Information:** Go beyond basic demographics to explore customer lifestyles, values, and habits. 

For instance, are there large groups of environmentally conscious consumers in your area who might appreciate eco-friendly laundry solutions?

In [2]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
from census import Census
from us import states

# Created modules
from ipython_config import CENSUS_KEY


In [ ]:
stats = pd.read_excel('data/us_state_6digitnaics_2022.xlsx')

In [4]:
stats.shape

(570107, 13)

In [5]:
stats.head()

,"The Number of Firms and Establishments, Employment, Annual Payroll, and Receipts by State, Industry, and Enterprise Employment Size: 2022",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Release Date: 4/10/2025,Unnamed: 12
0,SOURCE: 2022 County Business Patterns and Econ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Employment Noise Flag,"Annual Payroll\n($1,000)",Annual Payroll Noise Flag,"Receipts\n($1,000)",Receipts\nNoise Flag
2,00,United States,--,Total,01: Total,6395635,8298562,135748407,G,8965035263,G,50848996830,G
3,00,United States,--,Total,02: <5 employees,4029041,4035233,6295615,G,354954498,G,2084288587,G
4,00,United States,--,Total,03: 5-9 employees,1034135,1047874,6810300,G,311979624,G,1632588564,G


In [6]:
stats.columns = stats.iloc[1]

In [7]:
stats = stats.iloc[2:]

In [8]:
laudromat = stats[(stats['NAICS'] == '812310') & (stats['State'] != '00')]

In [9]:
laudromat[laudromat['State Name'] == 'Pennsylvania']

1,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Employment Noise Flag,"Annual Payroll\n($1,000)",Annual Payroll Noise Flag,"Receipts\n($1,000)",Receipts\nNoise Flag
445437,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,01: Total,282,311,1152,G,28856,G,142294,G
445438,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,02: <5 employees,204,206,354,G,6618,G,38563,G
445439,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,03: 5-9 employees,50,52,314,G,5169,G,27384,G
445440,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,04: 10-19 employees,18,18,191,G,4883,G,15378,G
445441,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,05: <20 employees,272,276,859,G,16670,G,81325,G
445442,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,06: 20-99 employees,9,32,244,H,8860,H,43380,H
445443,42,Pennsylvania,812310,Coin-Operated Laundries and Drycleaners,08: <500 employees,281,308,1103,G,25530,G,124705,G


In [10]:
# Set API key
c = Census(CENSUS_KEY)

In [11]:
# Obtain Census variables from the 2019 ACS at the tract level for the Commonwealth of Virginia (FIPS code: 51)
# C17002_001E: count of ratio of income to poverty in the past 12 months (total)
# C17002_002E: count of ratio of income to poverty in the past 12 months (< 0.50)
# C17002_003E: count of ratio of income to poverty in the past 12 months (0.50 - 0.99)
# B01003_001E: total population
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
va_census = c.acs5.state_county_tract(fields = ('NAME', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B01003_001E'),
                                      state_fips = states.VA.fips,
                                      county_fips = "*",
                                      tract = "*",
                                      year = 2017)

In [12]:
# Create a dataframe from the census data
va_df = pd.DataFrame(va_census)

# Show the dataframe
print(va_df.head(2))
print('Shape: ', va_df.shape)

                                         NAME  C17002_001E  C17002_002E  \
0     Census Tract 60, Norfolk city, Virginia       3947.0        284.0   
1  Census Tract 65.02, Norfolk city, Virginia       3287.0        383.0   

   C17002_003E  B01003_001E state county   tract  
0        507.0       3947.0    51    710  006000  
1        480.0       3302.0    51    710  006502  
Shape:  (1907, 8)


In [13]:
# Combine state, county, and tract columns together to create a new string and assign to new column
va_df["GEOID"] = va_df["state"] + va_df["county"] + va_df["tract"]

In [14]:
# Remove columns
va_df = va_df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
va_df.head(2)

,NAME,C17002_001E,C17002_002E,C17002_003E,B01003_001E,GEOID
0,"Census Tract 60, Norfolk city, Virginia",3947.0,284.0,507.0,3947.0,51710006000
1,"Census Tract 65.02, Norfolk city, Virginia",3287.0,383.0,480.0,3302.0,51710006502


In [15]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(va_df.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

Column data types for census data:
NAME            object
C17002_001E    float64
C17002_002E    float64
C17002_003E    float64
B01003_001E    float64
GEOID           object
dtype: object


In [16]:
# Get poverty rate and store values in new column
va_df["Poverty_Rate"] = (va_df["C17002_002E"] + va_df["C17002_003E"]) / va_df["B01003_001E"] * 100

# Show dataframe
va_df.head(2)

,NAME,C17002_001E,C17002_002E,C17002_003E,B01003_001E,GEOID,Poverty_Rate
0,"Census Tract 60, Norfolk city, Virginia",3947.0,284.0,507.0,3947.0,51710006000,20.040537
1,"Census Tract 65.02, Norfolk city, Virginia",3287.0,383.0,480.0,3302.0,51710006502,26.135675


api.census.gov/data/2022/abscbo?get=GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F&for=us:*&QDESC_LABEL=YRACQBUS&key=YOUR_KEY_GOES_HERE

In [38]:
import requests

def get_abs_data(variables, geography="us:*", api_key=None):
    """
    Get data from the Annual Business Survey (ABS) API
    
    Parameters:
    - variables: Comma-separated string of variable names
    - geography: Geographic level (default: national level)
    - api_key: Your Census Bureau API key
    """
    base_url = "https://api.census.gov/data"
    year = "/2022"
    dataset = "/abscbo"
    
    # Construct the full URL
    url = f"{base_url}{year}{dataset}?get={variables}&for={geography}"
    
    if api_key:
        url += f"&key={api_key}"
        
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve data. Status code: {response}")

# Example usage
try:
    # Get basic business statistics nationally
    data = get_abs_data("GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F", 
                        api_key=CENSUS_KEY, 
                        geography='state:*&QDESC_LABEL=YRACQBUS')
except Exception as e:
    print(f"Error: {str(e)}")

In [39]:
usb = pd.DataFrame(data)
usb.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,...,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F,QDESC_LABEL,state
1,0400000US01,Alabama,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,...,1324,None,2.7,None,11.9,None,0.4,None,YRACQBUS,01
2,0400000US01,Alabama,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,...,2694,None,5.5,None,6.6,None,0.5,None,YRACQBUS,01
3,0400000US01,Alabama,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,...,5493,None,11.2,None,8.7,None,1.0,None,YRACQBUS,01
4,0400000US01,Alabama,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,00,All owners of respondent firms,...,9781,None,19.9,None,3.5,None,0.7,None,YRACQBUS,01


In [42]:
usb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10548 entries, 0 to 10547
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       10548 non-null  object
 1   1       10548 non-null  object
 2   2       10548 non-null  object
 3   3       10548 non-null  object
 4   4       10548 non-null  object
 5   5       10548 non-null  object
 6   6       10548 non-null  object
 7   7       10548 non-null  object
 8   8       10548 non-null  object
 9   9       10548 non-null  object
 10  10      10548 non-null  object
 11  11      10548 non-null  object
 12  12      10548 non-null  object
 13  13      10548 non-null  object
 14  14      10548 non-null  object
 15  15      10548 non-null  object
 16  16      10548 non-null  object
 17  17      10548 non-null  object
 18  18      833 non-null    object
 19  19      10548 non-null  object
 20  20      1380 non-null   object
 21  21      10548 non-null  object
 22  22      833 non-null  

In [41]:
usb[3].value_counts()

3
Total for all sectors                                                       561
Retail trade                                                                560
Professional, scientific, and technical services                            560
Accommodation and food services                                             558
Construction                                                                556
Manufacturing                                                               554
Administrative and support and waste management and remediation services    551
Wholesale trade                                                             550
Real estate and rental and leasing                                          550
Finance and insurance                                                       549
Other services (except public administration)                               549
Health care and social assistance                                           549
Transportation and warehousing        

In [31]:
usb[3].value_counts()

3
Total for all sectors    28519
NAICS2022_LABEL              1
Name: count, dtype: int64

## 2. Research Potential Locations:
- **Demographics:** Research the demographics of potential areas to ensure they align with your target market
- **Competition:** Analyze the competitive landscape in each area to understand the level of competition
- **Infrastructure:** Assess the availability of essential infrastructure, such as 
    - transportation
    - utilities
    - internet access
- **Zoning Laws:** Understand local zoning regulations and restrictions to ensure compliance
- **Local Resources:** Explore local government census data or tools like WIGeoLocation for further research  
- **Accessibility:** Evaluate the accessibility of the location for both customers and employees
- **Traffic:** Consider traffic patterns and parking availability, especially if foot traffic is important 

## 3. Evaluate and Compare:  
- **Visit Potential Locations:** Conduct site visits to assess the suitability of each location firsthand
- **Cost Analysis:** Compare the costs of different 
    - locations
    - including rent
    - utilities
    - taxes  
- **Pros and Cons:** Create a list of pros and cons for each potential location to help make a decision
- **Long-Term Growth Potential:** Consider the long-term growth potential of the area and its ability to support your business  

## 4. Consider Legal and Institutional Factors: 
- **Business Licenses:** Research the necessary business licenses and permits required for your business type and location. 
- **Local Regulations:** Understand any local regulations or restrictions that may affect your business operations.
- **Government Incentives:** Explore any local or state government incentives or programs that may be available for businesses in specific areas.